In [ ]:
## Minimal 2-qubit QBM

A Quantum Born Machine (QBM) is a generative quantum machine learning model that learns probability distributions using the Born rule. In essence, the QBM is given a target distribution and must adjust its quantum circuit parameters so that the distribution of its measurement outcomes becomes similar to the given one. This lab demonstrates the smallest non-trivial QBM, built with two qubits and a single variational parameter.

![2-qubit-QBM](images/2-qubit-QBM.png)

The model circuit consists of:

1. An `RY(θ)` rotation on qubit 0.  
2. A `CX(0→1)` gate to entangle the two qubits.  
3. Measurement in the computational basis.  

The resulting state is:

$$
\cos\!\left(\tfrac{\theta}{2}\right)\lvert 00\rangle \;+\; \sin\!\left(\tfrac{\theta}{2}\right)\lvert 11\rangle
$$

Hence, measurement outcomes are restricted to:

$$
P_{\text{model}}(00) = \cos^2\!\left(\tfrac{\theta}{2}\right), \qquad
P_{\text{model}}(11) = \sin^2\!\left(\tfrac{\theta}{2}\right).
$$

This makes the model capable of representing any binary distribution supported on $\{00,11\}$.

## Code components

- **`vqc(theta, plot=False)`**  
  Builds the variational quantum circuit, executes it on the simulator, and returns the measured probability distribution.  

- **Loss functions**  
  The goal is to compare the model distribution $p_{\text{model}}$ against the target distribution $p_{\text{target}}$.  

  - `l1_loss(p_model, t_dist)`  
    Computes the $\ell_1$ (Manhattan) distance:  
    $$
    L_1(p_{\text{model}}, p_{\text{target}}) \;=\; 
    \sum_{x} \big|\, p_{\text{model}}(x) - p_{\text{target}}(x)\,\big|
    $$
  
  - `l2_loss(p_model, t_dist)`  
    Computes the $\ell_2$ (Euclidean) distance:  
    $$
    L_2(p_{\text{model}}, p_{\text{target}}) \;=\;
    \sqrt{\;\sum_{x} \big( p_{\text{model}}(x) - p_{\text{target}}(x) \big)^2}
    $$

- **`basic_optimizer(loss_fn, t_dist, step_size, max_iter)`**  
  Performs a naive optimization loop:  
  - Starts at $\theta = 0$.  
  - Increases $\theta$ step by step.  
  - Evaluates the model distribution and its loss compared to the target.  
  - Stops when the loss starts to increase.  

---

## Task

1. Run the notebook to optimize $\theta$ for the default target distribution:  
   `t_dist = {'00': 0.3, '11': 0.7}`  

2. Observe how the optimizer adjusts $\theta$ to minimize the loss.  
3. Replace `t_dist` with other valid probability distributions on $\{00,11\}$ (for example, `{'00': 0.5, '11': 0.5}`) and compare results.  
4. Try both `l1_loss` and `l2_loss` to see how they behave differently.  

## Expected output

- A loss vs. $\theta$ plot showing how the loss decreases as the parameter is optimized.  
- A circuit diagram of the final trained QBM.  
- Overlayed bar plots of $p_{\text{model}}$ and $t_{\text{dist}}$ for a quick visual check of the fit. 
- A printout of the optimized parameter value and the number of iterations.  

## Experimentation

- Extend the circuit so it can also place probability on $\lvert 01\rangle$ and $\lvert 10\rangle$.  
- Replace the naive line search with a simple gradient-based update on $\theta$.  
- Study how different shot counts affect the learned $\theta$ and final loss.  


In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_distribution
from qiskit.visualization import circuit_drawer
import matplotlib.pyplot as plt
import numpy as np


def vqc(theta, plot=False):
    """
    Creates and runs a simple 2-qubit variational quantum circuit.

    Applies an RY rotation by angle `theta` on qubit 0 followed by a CNOT gate to entangle qubit 0 and 1.
    The circuit is measured, optionally plotted, and executed on a simulator.

    Args:
        theta (float): Rotation angle for the RY gate on qubit 0.
        plot (bool): If True, displays the circuit diagram.

    Returns:
        dict: Probability distribution of measurement outcomes (bitstrings).
    """
    qc = QuantumCircuit(2,2)
    
    # set state       
    qc.ry(theta, 0)
    qc.cx(0, 1)

    qc.barrier()
    qc.measure(range(2), range(2))
    if plot:
        display(circuit_drawer(qc, output="mpl"))
    
    # -- run the program on the simulator --
    simulator = AerSimulator()
    result = simulator.run(qc, shots=1000).result()
    counts = result.get_counts(qc)

    # convert counts to probability distribution
    total = sum(counts.values())
    p_model = {k: v / total for k, v in counts.items()}
    
    return p_model

def l1_loss(p_model, t_dist):
    """
    Computes the L1 loss (Manhattan distance) between two probability distributions.

    The L1 loss is the sum of absolute differences between corresponding probabilities 
    in the model and target distributions. Missing keys are treated as zero.

    Args:
        p_model (dict): The predicted/model probability distribution.
        t_dist (dict): The target/reference probability distribution.

    Returns:
        float: Total L1 loss between the two distributions.
    """
    keys = set(p_model.keys()).union(t_dist.keys())
    loss = 0.0
    for k in keys:
        model_prob = p_model.get(k, 0.0)    # use 0.0 if key not found
        target_prob = t_dist.get(k, 0.0)    # use 0.0 if key not found
        diff = abs(model_prob - target_prob)
        loss += diff
    return loss

def l2_loss(p_model, t_dist):
    """
    Computes the L2 loss (Euclidean distance) between two probability distributions.

    The L2 loss is the square root of the sum of squared differences between corresponding 
    probabilities in the model and target distributions. Missing keys are treated as zero.

    Args:
        p_model (dict): The predicted/model probability distribution.
        t_dist (dict): The target/reference probability distribution.

    Returns:
        float: Total L2 loss between the two distributions.
    """

    keys = set(p_model.keys()).union(t_dist.keys())
    loss = 0.0
    for k in keys:
        model_prob = p_model.get(k, 0.0)    # use 0.0 if key not found
        target_prob = t_dist.get(k, 0.0)    # use 0.0 if key not found
        diff = model_prob - target_prob
        loss += diff ** 2
    return loss ** 0.5   # square root for L2


def basic_optimizer(loss_fn, t_dist, step_size=0.1, max_iter=100):
    """
    Optimizes the rotation angle theta to minimize the loss 
    between the VQC output distribution and a target distribution.

    The function incrementally increases theta, evaluates the VQC output, 
    and computes the loss against the target. Optimization stops when 
    the loss increases, indicating a minimum has been passed.

    Args:
        loss_fn: loss function name
        t_dist (dict): Target probability distribution.
        step_size (float): Increment for theta in each iteration.
        max_iter (int): Maximum number of optimization steps.

    Returns:
        tuple: Lists of theta values and corresponding losses.
    """
    theta = 0.0
    losses = []
    thetas = []

    prev_loss = None

    for i in range(max_iter):
        # get model distribution from VQC at current theta
        p_model = vqc(theta)

        # compute L1 loss
        loss = loss_fn(p_model, t_dist)

        # check if loss starts increasing
        if prev_loss is not None and loss > prev_loss:
            losses.pop() # discard the last guess since it performed worse 
            thetas.pop()
            break
            
        # store for graphing
        thetas.append(theta)
        losses.append(loss)

        prev_loss = loss
        theta += step_size

    return thetas, losses, i

# ------------------------------------------------
#                 main program
# ------------------------------------------------
# target distribution
t_dist = {'00': 0.3, '11': 0.7} 
thetas, losses, iter = basic_optimizer(l2_loss, t_dist)

# plot results
plt.plot(thetas, losses, marker='o')
plt.xlabel("θ")
plt.ylabel("L2 Loss")
plt.title("Loss vs θ")
plt.grid(True)
plt.show()

# display circuit and results with final paramter
model_dist = vqc(thetas[-1], True)
display(
    plot_distribution(
        [t_dist, model_dist],
        title="Target vs Model Distribution",
        legend=["Target (requested)", "Model"]
    )
)
print(f"Optimization finished after {iter} iterations with θ = {thetas[-1]}")
